In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import time

import tensorflow as tf
tf.random.set_seed(42)

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / max(y_true))) * 100

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
# Specifying the path of file
path= r"C:\Users\sashah8\OneDrive - North Carolina State University\Desktop\Q4\Data\L2\data_cleaned"

# Listing just the required files
fileList = os.listdir(path)
fileList.remove("Date.csv")

# Previewing the required file names
print(fileList)

['COAST.csv', 'EAST.csv', 'FWEST.csv', 'NCENT.csv', 'NORTH.csv', 'SCENT.csv', 'SOUTH.csv', 'WEST.csv']


In [3]:
# Just for previewing the columns
pd.read_csv(os.path.join(path, fileList[4]))

,Net,Temprature
0,901.770195,-8.8
1,890.150373,-8.8
2,880.926639,-8.8
3,883.990576,-8.7
4,894.320303,-8.6
...,...,...
43818,825.761799,9.2
43819,822.190573,9.9
43820,819.394012,10.0
43821,804.714775,9.2


### PROCESSING THE DATA

In [4]:
# Restricting the load data to a single zone
fileName  = fileList[4]

zoneData = pd.read_csv(os.path.join(path, fileName))
zoneData.columns = ["Load_"+str(fileName[:-4]), "Temp_"+str(fileName[:-4])]

# For fixing one missing row
zoneData = pd.concat([zoneData.loc[[0]], zoneData], axis= 0).reset_index(drop= True)

zoneData

,Load_NORTH,Temp_NORTH
0,901.770195,-8.8
1,901.770195,-8.8
2,890.150373,-8.8
3,880.926639,-8.8
4,883.990576,-8.7
...,...,...
43819,825.761799,9.2
43820,822.190573,9.9
43821,819.394012,10.0
43822,804.714775,9.2


### FEATURE ENGINEERING AND SPLITTING INTO TRAIN-TEST SPLIT

In [5]:
def lagGenerator(dF, lagList):
    dF_temp = dF.filter(regex= "Load").copy()
    
    for i in lagList:
        dF_temp.loc[:, "Lag-" + str(i)] = dF_temp.iloc[:,[0]].shift(i*24)
    
    return(dF_temp.filter(regex= "Lag"))

In [6]:
Load_lags = lagGenerator(dF=zoneData, lagList=[2,7])
Load_lags.head(2)

,Lag-2,Lag-7
0,NaN,NaN
1,NaN,NaN


In [7]:
dataDf = pd.concat([zoneData, Load_lags], axis= 1).dropna(axis= 0).reset_index(drop= True)
dataDf

,Load_NORTH,Temp_NORTH,Lag-2,Lag-7
0,1085.239500,-5.5,888.111250,901.770195
1,1069.995045,-5.6,863.256668,901.770195
2,1074.042161,-5.9,861.841143,890.150373
3,1076.248185,-6.1,866.678923,880.926639
4,1089.438363,-6.2,879.735020,883.990576
...,...,...,...,...
43651,825.761799,9.2,865.670716,720.820842
43652,822.190573,9.9,864.507781,719.864419
43653,819.394012,10.0,850.149917,714.949676
43654,804.714775,9.2,821.959349,698.896222


In [8]:
# Setting the train-test split ratio
ratio = 0.1

In [9]:
Num_train, Num_test = int(len(dataDf)/24 * (1 - ratio)) * 24, int(len(dataDf)/24 * ratio) * 24

In [10]:
# Splitting the data into train and test
data_train, data_test = dataDf[:Num_train], dataDf[Num_train:]

In [11]:
Num_train, Num_valid = int(len(data_train)/24 * (1 - ratio)) * 24, int(len(data_train)/24 * ratio) * 24

In [12]:
# Splitting the data into train and test
data_train, data_valid = data_train[:Num_train], data_train[Num_train:]

### SPLITTING INTO X AND Y PART WITH SEQUENCE BATCHING

In [13]:
X_train_df, Y_train_df = data_train.iloc[:, 1:], data_train.iloc[:, :1]

X_valid_df, Y_valid_df = data_valid.iloc[:, 1:], data_valid.iloc[:, :1]

X_test_df, Y_test_df = data_test.iloc[:, 1:], data_test.iloc[:, :1]

In [14]:
# Preparing the scaler
ValScaler = MinMaxScaler()

X_train = ValScaler.fit_transform(X_train_df)
X_valid = ValScaler.transform(X_valid_df)
X_test = ValScaler.transform(X_test_df)

Y_train = ValScaler.fit_transform(Y_train_df)
Y_valid = ValScaler.transform(Y_valid_df)
Y_test = ValScaler.transform(Y_test_df)

In [15]:
def splitData(dataset):
    data = np.array(np.split(dataset, len(dataset)/24))
    return data

In [16]:
def supervisedSet(X_part, Y_part, steps= 24):
    X_array, Y_array = np.array(X_part), np.array(Y_part)
    X, Y = [], []
    
    for i in range(len(X_array)):
        endIdx = i + steps
        
        if endIdx > len(X_array):
            break
        
        batch_X, batch_Y = X_array[i:endIdx], Y_array[i:endIdx]
        
        X.append(batch_X)
        Y.append(batch_Y)
        
    return np.array(X), np.array(Y)

In [17]:
X_train, Y_train = supervisedSet(X_train, Y_train)

X_valid, Y_valid = splitData(X_valid), splitData(Y_valid)

X_test, Y_test = splitData(X_test), splitData(Y_test)

### WORKING ON THE MODEL

In [18]:
epochs = 100
batchSize = 32

In [19]:
# Encoder-decoder architecture [tanh required to use tensorflow on gpu]
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(units= 128, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2]), dropout= 0.2),
    tf.keras.layers.RepeatVector(Y_train.shape[1]),
    tf.keras.layers.LSTM(units= 128, activation= "tanh", return_sequences= True, dropout= 0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 128, activation= "relu")),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units= 1))
])

model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [20]:
# model = tf.keras.Sequential([
#     tf.keras.layers.LSTM(units= 256, activation= "tanh", input_shape= (X_train.shape[1], X_train.shape[2])),
#     tf.keras.layers.Dense(units= 128, activation= "tanh"),
#     tf.keras.layers.Dense(units= 1)
# ])

# model.compile(loss= "mse", optimizer= "adam",  metrics= ['mape', 'mae'])

In [21]:
# Storing the start time of model fitting process
startTime = time.time()

In [22]:
model.fit(X_train, Y_train, epochs= epochs, batch_size= batchSize, verbose= 1, shuffle= True, validation_data= (X_valid, Y_valid))

Epoch 1/100
1105/1105 [==============================] - 27s 17ms/step - loss: 0.0121 - mape: 3710.9709 - mae: 0.0823 - val_loss: 0.0093 - val_mape: 36.6731 - val_mae: 0.0717
Epoch 2/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0076 - mape: 3050.6287 - mae: 0.0662 - val_loss: 0.0102 - val_mape: 34.6289 - val_mae: 0.0759
Epoch 3/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0059 - mape: 2496.2881 - mae: 0.0580 - val_loss: 0.0109 - val_mape: 38.4694 - val_mae: 0.0799
Epoch 4/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0054 - mape: 2430.1772 - mae: 0.0552 - val_loss: 0.0093 - val_mape: 38.0810 - val_mae: 0.0762
Epoch 5/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0051 - mape: 2143.1794 - mae: 0.0539 - val_loss: 0.0127 - val_mape: 33.8720 - val_mae: 0.0828
Epoch 6/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0049 - mape: 2383.1802 - mae: 0.0530 - val_l

1105/1105 [==============================] - 18s 16ms/step - loss: 0.0024 - mape: 2003.4917 - mae: 0.0365 - val_loss: 0.0123 - val_mape: 30.1719 - val_mae: 0.0808
Epoch 48/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0024 - mape: 2096.0593 - mae: 0.0362 - val_loss: 0.0137 - val_mape: 33.6192 - val_mae: 0.0873
Epoch 49/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0023 - mape: 1834.3279 - mae: 0.0360 - val_loss: 0.0130 - val_mape: 32.1815 - val_mae: 0.0848
Epoch 50/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0023 - mape: 1838.7505 - mae: 0.0358 - val_loss: 0.0117 - val_mape: 30.9489 - val_mae: 0.0794
Epoch 51/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0022 - mape: 1835.0452 - mae: 0.0353 - val_loss: 0.0131 - val_mape: 34.8785 - val_mae: 0.0867
Epoch 52/100
1105/1105 [==============================] - 17s 16ms/step - loss: 0.0022 - mape: 1806.6453 - mae: 0.0350 - val_loss: 0.

1105/1105 [==============================] - 18s 16ms/step - loss: 0.0012 - mape: 1613.3622 - mae: 0.0259 - val_loss: 0.0124 - val_mape: 31.4636 - val_mae: 0.0819
Epoch 94/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0012 - mape: 1552.4025 - mae: 0.0255 - val_loss: 0.0128 - val_mape: 32.3838 - val_mae: 0.0840
Epoch 95/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0011 - mape: 1799.5964 - mae: 0.0254 - val_loss: 0.0145 - val_mape: 35.0539 - val_mae: 0.0908
Epoch 96/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0012 - mape: 1809.0748 - mae: 0.0260 - val_loss: 0.0127 - val_mape: 32.6325 - val_mae: 0.0839
Epoch 97/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0011 - mape: 1744.5314 - mae: 0.0252 - val_loss: 0.0135 - val_mape: 33.5620 - val_mae: 0.0876
Epoch 98/100
1105/1105 [==============================] - 18s 16ms/step - loss: 0.0011 - mape: 1793.8409 - mae: 0.0254 - val_loss: 0.

In [23]:
calcExecTime = (time.time() - startTime)
print('Total execution time in seconds: ' + str(calcExecTime))

Total execution time in seconds: 1867.7108180522919


### MAKING PREDICTIONS AND GENERATING RESULTS

In [24]:
predictions = model.predict(X_test)

6/6 [==============================] - 1s 11ms/step


In [25]:
predArr = predictions.reshape(predictions.shape[0] * predictions.shape[1], predictions.shape[2]).ravel()
predArr

array([0.1618011 , 0.13752317, 0.12072557, ..., 0.4042502 , 0.37266988,
       0.3431908 ], dtype=float32)

In [26]:
Y_testArr = Y_test.reshape(Y_test.shape[0] * Y_test.shape[1], Y_test.shape[2]).ravel()
Y_testArr

array([0.30705154, 0.26919234, 0.24240777, ..., 0.29210277, 0.27730973,
       0.26335294])

In [27]:
def inverse_transform(df):
    column_names = df.columns
    for col in column_names:
        df[col] = ValScaler.inverse_transform(df[[col]]) 
    return df

In [28]:
comparisionDf = pd.DataFrame(data= {"Values": Y_testArr, "Predictions": predArr})
comparisionDf = inverse_transform(comparisionDf)
comparisionDf

,Values,Predictions
0,834.227772,690.094727
1,796.659830,666.003601
2,770.081317,649.335205
3,768.146601,643.297424
4,783.320723,655.694153
...,...,...
4363,825.761799,954.334167
4364,822.190573,951.434265
4365,819.394012,930.678650
4366,804.714775,899.341309


In [29]:
mape(comparisionDf["Values"], comparisionDf["Predictions"])

10.579628657903978